In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import numpy as np
from sklearn.pipeline import Pipeline
from sktime.classification.compose import ColumnEnsembleClassifier
from sktime.classification.compose import TimeSeriesForestClassifier
from sktime.classification.dictionary_based import BOSSEnsemble
from sktime.classification.shapelet_based import MrSEQLClassifier
from sktime.datasets import load_basic_motions
from sktime.transformers.series_as_features.compose import ColumnConcatenator
from sklearn.model_selection import train_test_split
import pandas as pd

In [ ]:
data_root = '/media/scottcha/E1/Data/Temp/Tutorial/'
ml_path = data_root + '/5.MLData/'
num_features = 978
interpolation = 1

In [ ]:
#load the full X datafile which has both Train and Test concated
X = np.load(ml_path + '/X_all.npy', mmap_mode='r')

In [ ]:
#read in the corresponding label files and concat them
#can get the right values here based on the contents of hte ml_path directory
num_y_batches = 1
num_y_train_files_per_batch = 20
num_y_test_batches = 1
num_y_test_files_per_batch = 1


y_df = None
for i in range(0, num_y_batches):
    for j in range(0, num_y_train_files_per_batch):
        df = pd.read_parquet(ml_path + '/y_train_batch_' + str(i) + '_' + str(j) + '.0.parquet')  
        y_df = pd.concat([y_df, df])

for i in range(0, num_y_test_batches):
    for j in range(0, num_y_test_files_per_batch):
        df = pd.read_parquet(ml_path + '/y_test_batch_' + str(i) + '_' + str(j) + '.0.parquet')  
        y_df = pd.concat([y_df, df])

In [ ]:
X.shape

(10500, 978, 180)

In [ ]:
#convert the labels to encoded values
y_df['Day1AboveCat'] = y_df['Day1DangerAboveTreeline'].astype('category')
y = y_df['Day1AboveCat'].cat.codes.values

In [ ]:
feature_means = np.load(ml_path + '/feature_means_interpolation1x.npy')
feature_means = (np.nan_to_num(feature_means))

In [ ]:
X2 = np.zeros_like(X)

In [ ]:
for i in range(X.shape[1]):
    X2[:,i,:] = np.nan_to_num(X[:,i,:], nan=feature_means[i])

In [ ]:
np.save(ml_path + 'X_noNan', X2)

In [ ]:
del X2

In [ ]:
X = np.load(ml_path + 'X_noNan.np.npy', mmap_mode='r')

In [ ]:
%%time
steps = [
    ('concatenate', ColumnConcatenator()),
    ('classify', TimeSeriesForestClassifier(n_estimators=25, n_jobs=4 ))]
clf = Pipeline(steps)
clf.fit(X[:10000,:,:], y[:10000])

In [ ]:
clf.score(X2[10000:,:,:], y[10000:])

0.154